New process - note that new wavenet test works much better.

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
####### new everything

frame_rate = 44100

import os
import soundfile as sf
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the audio files
def load_wav_file(file_path):
  data, frame_rate = sf.read(file_path)
  return data, frame_rate

# Function to save file
def save_wav_file(file_path, data, frame_rate):
 sf.write(file_path, data, frame_rate)

audio_folder = '/content/gdrive/MyDrive/Development/audio/'
di_train_path = os.path.join(audio_folder, 'train_x.wav')
dist_train_path = os.path.join(audio_folder, 'train_y.wav')
di_test_path = os.path.join(audio_folder, 'di_01.wav')

di_train_data, _ = load_wav_file(di_train_path)
dist_train_data, _ = load_wav_file(dist_train_path)
di_test_data, _ = load_wav_file(di_test_path)

# Reshape the data to 2D
di_train_data = np.expand_dims(di_train_data, axis=1)
dist_train_data = np.expand_dims(dist_train_data, axis=1)
di_test_data = np.expand_dims(di_test_data, axis=1)

# Build the model
model = Sequential()
model.add(Dense(256, input_shape=(di_train_data.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(dist_train_data.shape[1], activation='linear'))

model.compile(optimizer='adam', loss='mse')

In [4]:

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Define the model checkpoint callback
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, monitor='val_loss')


In [10]:


# Train the model with early stopping and model checkpoint
history = model.fit(di_train_data, dist_train_data, epochs=10, validation_split=0.2, callbacks=[early_stopping, checkpoint])




Epoch 1/10
264600/264600 [==============================] - 974s 4ms/step - loss: 0.0665 - val_loss: 0.0579
Epoch 2/10
264600/264600 [==============================] - 948s 4ms/step - loss: 0.0665 - val_loss: 0.0578
Epoch 3/10
264600/264600 [==============================] - 914s 3ms/step - loss: 0.0665 - val_loss: 0.0577
Epoch 4/10
264600/264600 [==============================] - 948s 4ms/step - loss: 0.0665 - val_loss: 0.0577
Epoch 5/10
264600/264600 [==============================] - 911s 3ms/step - loss: 0.0665 - val_loss: 0.0577
Epoch 6/10
264600/264600 [==============================] - 908s 3ms/step - loss: 0.0665 - val_loss: 0.0578
Epoch 7/10
264600/264600 [==============================] - 911s 3ms/step - loss: 0.0665 - val_loss: 0.0577


In [11]:
model.save('/content/gdrive/MyDrive/Development/audio/yet_another_amp_model.h5')


In [8]:
# optional - load saved model
# from tensorflow.keras.models import load_model
# load_model('/content/gdrive/MyDrive/Development/audio/yet_another_amp_model.h5')

In [20]:
# Predict the test output
dist_test_data = model.predict(di_test_data)



31798/31798 [==============================] - 203s 6ms/step


In [21]:
# Save the test output as a WAV file
dist_test_path = os.path.join(audio_folder, 'dist_testttt2.wav')
save_wav_file(dist_test_path, dist_test_data, frame_rate)

New wavenet test

In [ ]:
!pip install keras-tcn


In [14]:


frame_rate = 44100

import os
import soundfile as sf
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tcn import TCN


# Load the audio files
def load_wav_file(file_path):
  data, frame_rate = sf.read(file_path)
  return data, frame_rate

# Function to save file
def save_wav_file(file_path, data, frame_rate):
 sf.write(file_path, data, frame_rate)

audio_folder = '/content/gdrive/MyDrive/Development/audio/'
di_train_path = os.path.join(audio_folder, 'train_x.wav')
dist_train_path = os.path.join(audio_folder, 'train_y.wav')
di_test_path = os.path.join(audio_folder, 'di_01.wav')

di_train_data, _ = load_wav_file(di_train_path)
dist_train_data, _ = load_wav_file(dist_train_path)
di_test_data, _ = load_wav_file(di_test_path)

# Reshape the data to 2D
di_train_data = np.expand_dims(di_train_data, axis=1)
dist_train_data = np.expand_dims(dist_train_data, axis=1)
di_test_data = np.expand_dims(di_test_data, axis=1)

# Build the model
model = Sequential()
model.add(TCN(input_shape=(None, 1)))
model.add(Dropout(0.5))
model.add(Dense(dist_train_data.shape[1], activation='linear'))

model.compile(optimizer='adam', loss='mse')

In [16]:
# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Define the model checkpoint callback
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, monitor='val_loss')

In [18]:
history = model.fit(di_train_data, dist_train_data, epochs=1, validation_split=0.2, callbacks=[early_stopping, checkpoint])


264600/264600 [==============================] - 6763s 26ms/step - loss: 0.0663 - val_loss: 0.0575


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model.save('/content/gdrive/MyDrive/Development/audio/yet_another_amp_model_wavenet.h5')


Old process - REMOVE

In [ ]:
# remove

# Export - I think you need this for below

import soundfile as sf
frame_rate = 44100
def save_wav_file(file_path, data, frame_rate):
 sf.write(file_path, data, frame_rate)

# Save the test output as a WAV file
#dist_test_path = os.path.join(audio_folder, 'dist_test.wav')
save_wav_file('/content/gdrive/MyDrive/Development/audio/dist_testxxxxx.wav', dist_test_data, frame_rate)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# remove

import os
import wave
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
# remove
import soundfile as sf

def load_wav_file(file_path):
   data, frame_rate = sf.read(file_path)
   return data, frame_rate

di_train_data, _ = load_wav_file('/content/gdrive/MyDrive/Development/audio/train_x.wav')
dist_train_data, _ = load_wav_file('/content/gdrive/MyDrive/Development/audio/train_y.wav')
di_test_data, _ = load_wav_file('/content/gdrive/MyDrive/Development/audio/di_01.wav')


In [ ]:
# remove
#di_train_data = np.expand_dims(di_train_data, axis=1)
di_train_data = di_train_data.reshape(-1, 1)
dist_train_data = dist_train_data.reshape(-1, 1)


In [ ]:
# remove
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(di_train_data.shape[1],)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(dist_train_data.shape[1])
])



In [ ]:
# remove
model.compile(optimizer='adam', loss='mse')

history = model.fit(di_train_data, dist_train_data, epochs=10, validation_split=0.2)

In [ ]:
# remove
dist_test_data = model.predict(di_train_data[0:301753,])
# dist_test_data = model.predict(di_test_data)

9430/9430 [==============================] - 18s 2ms/step
